<a href="https://colab.research.google.com/github/jibin-sebastian/Deep-Learning-Recipes/blob/main/CNN_Model_HyperParameterOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Create CNN Model and Optimize it using Keras Tuner

In [3]:
!pip install keras-tuner

     |████████████████████████████████| 98 kB 3.6 MB/s 


In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
#Data Preparation stage
#load dataset
fashion_mnist=keras.datasets.fashion_mnist
#create train and test dataset
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
#scaled down the grey image between 0 and 255. Increase the calculation efficiancy
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
#Reshape the train and test data.
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
#reference : https://keras.io/guides/keras_tuner/getting_started/

def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
#import libraries 
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import kerastuner

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:

tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 48s]
val_accuracy: 0.906000018119812

Best val_accuracy So Far: 0.9166666865348816
Total elapsed time: 00h 05m 14s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 48)        76848     
                                                                 
 flatten (Flatten)           (None, 23232)             0         
                                                                 
 dense (Dense)               (None, 128)               2973824   
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 3,052,602
Trainable params: 3,052,602
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.1257 - accuracy: 0.9520 - val_loss: 0.2579 - val_accuracy: 0.9157
Epoch 5/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0875 - accuracy: 0.9673 - val_loss: 0.2791 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0593 - accuracy: 0.9779 - val_loss: 0.3355 - val_accuracy: 0.9183
Epoch 7/10
1688/1688 [==============================] - 17s 10ms/step - loss: 0.0433 - accuracy: 0.9839 - val_loss: 0.3844 - val_accuracy: 0.9123
Epoch 8/10
1688/1688 [==============================] - 16s 9ms/step - loss: 0.0333 - accuracy: 0.9878 - val_loss: 0.4468 - val_accuracy: 0.9110
Epoch 9/10
1688/1688 [==============================] - 15s 9ms/step - loss: 0.0275 - accuracy: 0.9907 - val_loss: 0.5049 - val_accuracy: 0.9142
Epoch 10/10
1688/1688 [==============================] - 16s 10ms/step - loss: 0.0228 - accuracy: 0.9921 - val_loss: 0.5045 - va